In [16]:
import random
import time
import speech_recognition as sr

def recogSpeechFromMicrophone(recognizer, microphone):
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("Recognizer invalid")
    if not isinstance(microphone, sr.Microphone):
        raise TypeError("Microphone invalid")
    
    #Manejamos el ruido de fondo 
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
        
    #Guardamos la respuesta del microfono
    response = {
        "success": True, #Si el request se pudo hacer o no
        "error": None, #Si ocurre algun error
        "transcription": None #None si no se ha podido hacer, string con la transcripcion 
    }
    
    #Hacemos el request y manejamos los posibles errores
    try:
        response["transcription"] = recognizer.recognize_google(audio, language= "es-ES")
    except sr.RequestError: #No tiene acceso a la api de google
        response["success"] = False
        response["error"] = "API unavailable"
    except sr.UnknownValueError:
        response["error"] = "Unable to recognise speech" #No se entiende lo que se dice
    
    return response
    

In [17]:
if __name__ == "__main__":
    WORDS =["manzana", "plátano", "uvas", "kiwi", "fresas", "arándanos", "frambuesa", "mango", "piña"]
    NUM_GUESS = 4
    PROMPT_LIMIT = 5
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()
    word = random.choice(WORDS)
    instructions = (
        "Estoy pensando en una de estas palabras: \n"
        "{words}\n"
        "Tienes {n} intentos para adivinarla \n"
    ).format(words = ", ".join(WORDS), n = NUM_GUESS)
    print(instructions)
    time.sleep(3) #Dejamos dormir durante tres segundos
    for i in range(NUM_GUESS):
        for j in range(PROMPT_LIMIT):
            print("Intento {}. Habla!".format(i+1))
            guess = recogSpeechFromMicrophone(recognizer, microphone)
            if guess["transcription"]:
                break
            if not guess["success"]:
                break
            print("No he entendido lo que has dicho. ¿Puedes repetir? \n")
        if guess["error"]:
            print("ERROR: {}".format(guess["error"]))
            break
        print("Has dicho: {}".format(guess["transcription"]))
        guess_is_correct = guess["transcription"].lower() == word.lower()
        more_attempts = i < NUM_GUESS - 1
        
        if guess_is_correct:
            print("Correcto: HAS GANADO!!! ESTA ERA LA PALABRA QUE ESTABA PENSANDO {}".format(word))
            break
        elif more_attempts:
            print("Incorrecto: INTENTALO DE NUEVO \n")
        else:
            print("GAME OVER: ESTABA PENSANDO EN {}".format(word))
            break
            
            
    

Estoy pensando en una de estas palabras: 
manzana, plátano, uvas, kiwi, fresas, arándanos, frambuesa, mango, piña
Tienes 4 intentos para adivinarla 

Intento 1. Habla!
Has dicho: plátano y las patatas ni el refresco qué crees que ha pasado que la hamburguesa es mejor
Incorrecto: INTENTALO DE NUEVO 

Intento 2. Habla!
Has dicho: uvas
Incorrecto: INTENTALO DE NUEVO 

Intento 3. Habla!
Has dicho: kiwi
Incorrecto: INTENTALO DE NUEVO 

Intento 4. Habla!
Has dicho: mango
Correcto: HAS GANADO!!! ESTA ERA LA PALABRA QUE ESTABA PENSANDO mango
